In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import requests
from  bs4 import BeautifulSoup
import re
import time
import os
# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

- 나무위키를 html로가져와서 konlpy로 분석을 한다

### 최근 변경 내용 가져오기

In [2]:
# 주소
site = 'https://namu.wiki/RecentChanges'
# 요청한다
res = requests.get(site)
html = res.text
soup = BeautifulSoup(html,'lxml')
soup

<!DOCTYPE html>
<html data-n-head-ssr=""><head><title>최근 변경내역 - 나무위키</title><meta charset="utf-8" data-n-head="ssr"/><meta content="user-scalable=no, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0, width=device-width" data-n-head="ssr" name="viewport"/><meta content="ie=edge" data-n-head="ssr" http-equv="x-ua-compatible"/><meta content="the seed" data-n-head="ssr" name="generator"/><meta content="yes" data-n-head="ssr" name="mobile-web-app-capable"/><meta content="나무위키" data-n-head="ssr" name="application-name"/><meta content="나무위키" data-n-head="ssr" name="msapplication-tooltip"/><meta content="/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EB%8C%80%EB%AC%B8" data-n-head="ssr" name="msapplication-starturl"/><meta content="#008275" data-n-head="ssr" name="theme-color"/><meta content="noarchive" data-n-head="ssr" name="googlebot"/><link data-n-head="ssr" href="https://namu.wiki/RecentChanges" rel="canonical"/><link data-n-head="ssr" href="/opensearch.xml" rel="search" title="나무위키" typ

In [3]:
# 데이터를 수집한다
table_tag = soup.select_one('#app > div > div:nth-child(2) > article > div > table')
tbody_tag = table_tag.select_one('tbody')
tr_tag_list = tbody_tag.select('tr')
tr_tag_list

[<tr data-v-3e749e98=""><td data-v-3e749e98=""><a data-v-3e749e98="" href="/w/%EC%BD%94%EB%A5%B4%ED%82%A4">코르키</a> <a data-v-3e749e98="" href="/history/%EC%BD%94%EB%A5%B4%ED%82%A4">[역사]</a> <a data-v-3e749e98="" href="/diff/%EC%BD%94%EB%A5%B4%ED%82%A4?rev=1989&amp;oldrev=1988">[비교]</a> <a data-v-3e749e98="" href="/discuss/%EC%BD%94%EB%A5%B4%ED%82%A4">[토론]</a> <span data-v-3e749e98="">(<span class="m" data-v-3e749e98="" data-v-3ee54395="">-273</span>)</span></td> <td data-v-3e749e98=""><div class="v-popover" data-v-3e749e98="" data-v-e473dc5a=""><div aria-describedby="popover_63va8sencw" class="trigger" style="display:inline-block;"><a data-v-e473dc5a="">114.30.148.166</a> </div> <div aria-hidden="true" class="tooltip popover vue-popover-theme" id="popover_63va8sencw" style="visibility:hidden;display:none;" tabindex="0"><div class="wrapper"><div class="tooltip-inner popover-inner" style="position:relative;"><div><div data-v-e473dc5a=""><a data-v-52293496="" data-v-e473dc5a="" href="/con

In [4]:
# 링크 주소들을 담을 리스트
link_list = []
# 위에서 수집한 tr태그의 수 만큼 반보한다
for tr_tag in tr_tag_list:
    # print(tr_tag)
    # tr태그 내에서 td태그들을 가져온다
    td_list = tr_tag.select('td')
    # 추출한 td태그중 첫번째 td만 사용한다
    td_tag = td_list[0]
    # td_tag내의 a태그들을 가져온다
    a_tag_list = td_tag.select('a')
    # 가져온 a태그가 안에 들어있을경우에만 작업을 한다
    if len(a_tag_list) > 0:
        # 첫번째 a태그의 href속성값을 가져온다
        href = a_tag_list[0].get('href')
        link_list.append(href)
    
link_list

['/w/%EC%BD%94%EB%A5%B4%ED%82%A4',
 '/w/Ichika%20Nito',
 '/w/%EB%A7%88%ED%9D%90%EB%94%94%20%EC%9A%B4%EB%8F%99',
 '/w/%EC%8A%88%ED%8D%BC%EC%BD%98',
 '/w/%EA%B9%80%EB%8F%99%ED%98%84',
 '/w/%EC%82%BC%EA%B0%81%ED%95%A8%EC%88%98%EC%9D%98%20%EB%85%B8%EB%9E%98',
 '/w/%EC%9D%B4%EC%9D%80%ED%83%9D(%EC%B9%B4%ED%8A%B8%EB%9D%BC%EC%9D%B4%EB%8D%94%20%ED%94%84%EB%A1%9C%EA%B2%8C%EC%9D%B4%EB%A8%B8)',
 '/w/%EA%B8%B0%EB%8F%99%EC%A0%84%EC%82%AC%20%EA%B1%B4%EB%8B%B4%20SEED%20DESTINY',
 '/w/%EB%85%B8%EC%A7%84%EA%B5%AC',
 '/w/%EC%8A%A4%ED%83%80%ED%81%AC%EB%9E%98%ED%94%84%ED%8A%B8%202:%20%EA%B3%B5%ED%97%88%EC%9D%98%20%EC%9C%A0%EC%82%B0/%ED%98%91%EB%8F%99%EC%A0%84%20%EC%9E%84%EB%AC%B4/%EB%8F%8C%EC%97%B0%EB%B3%80%EC%9D%B4%EC%9B%90/%EB%B6%88%EA%BD%83%EB%86%80%EC%9D%B4',
 '/w/%EA%B0%95%EC%84%B1%ED%98%95',
 '/w/2021%EB%85%84%20%EB%A1%9C%EC%8A%A4%ED%8A%B8%EC%95%84%ED%81%AC%20%EC%9D%B4%EC%A3%BC%20%EB%8C%80%EB%9E%80',
 '/w/%EC%95%84%EC%9D%B4%EC%9C%A0%EB%B8%8C%20%EC%99%95%EC%A1%B0',
 '/w/%ED%8C%90%EC%82%AC',
 '/w/%EB%B

In [5]:
# 주소의 수만큼 반복한다
    # 페이지 내의 데이터를 수집한다
for sub_url in link_list:
    time.sleep(1)
    # 데이터를 담을 딕셔너리
    data_dict = {
    'title':[],
    'content':[]
    }
    # 요청주소 
    site2 = f'https://namu.wiki{sub_url}'
    print(f'{site2}수집중')
    # print(site2)
    # 요청한다
    res2 = requests.get(site2)
    soup2 = BeautifulSoup(res2.text,'lxml')
    # 글전체를 가져온다
    article_tag = soup2.select_one('#app > div > div:nth-child(2) > article')
    # 제목을 가져온다
    h1_tag = article_tag.select_one('h1')
    title_text = h1_tag.text.strip()
    title_text
    # 글 내용을 가저온다
    content_paragraphs = article_tag.select('div.wiki-paragraph')

    # 글 내용을 하나의 문자열로 합친다
    content_text = ''
    for content_tag in content_paragraphs:
        content_text = content_text + content_tag.text

    content_text

    # 제거 하고자 하는 문자열들을 제거한다
    content_text = content_text.replace('번 문단','')
    content_text = content_text.strip()
    # 글제목
    title_text = h1_tag.text.strip()
    # 데이터를 담는다
    data_dict['title'].append(title_text)
    data_dict['content'].append(content_text)
    
    # 데이터프레임 생성
    df1 = pd.DataFrame(data_dict)
    
    if os.path.exists('wiki.csv') ==False:
        df1.to_csv('wiki.csv',encoding = 'utf-8-sig',index = False)
    else:
        df1.to_csv('wiki.csv',encoding = 'utf-8-sig',index = False, header = False, mode = 'a')

https://namu.wiki/w/%EC%BD%94%EB%A5%B4%ED%82%A4수집중
https://namu.wiki/w/Ichika%20Nito수집중
https://namu.wiki/w/%EB%A7%88%ED%9D%90%EB%94%94%20%EC%9A%B4%EB%8F%99수집중
https://namu.wiki/w/%EC%8A%88%ED%8D%BC%EC%BD%98수집중
https://namu.wiki/w/%EA%B9%80%EB%8F%99%ED%98%84수집중
https://namu.wiki/w/%EC%82%BC%EA%B0%81%ED%95%A8%EC%88%98%EC%9D%98%20%EB%85%B8%EB%9E%98수집중
https://namu.wiki/w/%EC%9D%B4%EC%9D%80%ED%83%9D(%EC%B9%B4%ED%8A%B8%EB%9D%BC%EC%9D%B4%EB%8D%94%20%ED%94%84%EB%A1%9C%EA%B2%8C%EC%9D%B4%EB%A8%B8)수집중
https://namu.wiki/w/%EA%B8%B0%EB%8F%99%EC%A0%84%EC%82%AC%20%EA%B1%B4%EB%8B%B4%20SEED%20DESTINY수집중
https://namu.wiki/w/%EB%85%B8%EC%A7%84%EA%B5%AC수집중
https://namu.wiki/w/%EC%8A%A4%ED%83%80%ED%81%AC%EB%9E%98%ED%94%84%ED%8A%B8%202:%20%EA%B3%B5%ED%97%88%EC%9D%98%20%EC%9C%A0%EC%82%B0/%ED%98%91%EB%8F%99%EC%A0%84%20%EC%9E%84%EB%AC%B4/%EB%8F%8C%EC%97%B0%EB%B3%80%EC%9D%B4%EC%9B%90/%EB%B6%88%EA%BD%83%EB%86%80%EC%9D%B4수집중
https://namu.wiki/w/%EA%B0%95%EC%84%B1%ED%98%95수집중
https://namu.wiki/w/2021%EB%85%84%20

In [6]:
wiki_df = pd.read_csv('wiki.csv')
wiki_df

,title,content
0,코르키,리그 오브 레전드 챔피언 목록 주 역할군 내 가나다순 정렬 [ 암살자 ] ...
1,Ichika Nito,Ichika Nito 이름いちか にと (이치카 니토)출생1994년 7월 7일 (26...
2,마흐디 운동,1. 개요2. 배경3. 진행19세기 이집트의 케디브 왕조와 영국의 이중 지배를 받던...
3,슈퍼콘,은(는) 여기로 연결됩니다. 현대카드 슈퍼콘서트에 대한 내용은 현대카드/문화사업 문...
4,김동현,은(는) 여기로 연결됩니다. 1981년생 종합격투기 선수 겸 예능인 김동현에 대한 ...
...,...,...
91,괴수고질라,썰레임 님을존경하는분이다팬아트를 계속그릴거라고 한다[1]이유튜버를욕하지마세요...h...
92,이해진,[ 펼치기 · 접기 ]주요 계열사네이버 | A홀딩스 (Z홀딩스 · LINE · 야후...
93,로스크바(슈퍼스트링),이 문서에 스포일러가 포함되어 있습니다.이 문서가 설명하는 작품이나 인물 등에 대한...
94,2019년,로그인 후 편집 가능한 문서입니다.해2016년2017년2018년2019년2020년2...
